In [1]:
!pip install gensim==3.8.3
!pip install keras --upgrade
!pip install pandas --upgrade
!pip install tensorflow --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

In [3]:
vocab_size = 32965
# WORD2VEC 
W2V_SIZE = 300
SEQUENCE_LENGTH = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

In [4]:
# DATASET
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
TRAIN_SIZE = 0.8

# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 10
BATCH_SIZE = 1024

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)

# EXPORT
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.w2v"
TOKENIZER_MODEL = "tokenizer.pkl"
ENCODER_MODEL = "encoder.pkl"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#LOADING
load_dir = '/content/drive/MyDrive/nns/'
train_test_dir = load_dir+'saved_train_test/'

embedding_matrix = np.load(load_dir+'embedding_matrix.npy')

x_train = np.load(train_test_dir+'x_train.npy')
y_train = np.load(train_test_dir+'y_train.npy')

x_test = np.load(train_test_dir+'x_test.npy')
y_test = np.load(train_test_dir+'y_test.npy')

In [7]:
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print()
print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train (40000, 300)
y_train (40000, 1)

x_test (10000, 300)
y_test (10000, 1)


###NN Model

In [8]:
#HYPER PARAMETERS
model_name = "RNN"
num_epochs = 10
batch_size = 1024
rnn_units = 512
sequence_length=300
activation="sigmoid"
optimizer='adam'

In [9]:
#MODEL
embedding_layer = Embedding(vocab_size, W2V_SIZE, weights=[embedding_matrix], 
                            input_length=SEQUENCE_LENGTH, 
                            trainable=False)

model = Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.GRU(rnn_units,
                        recurrent_initializer='glorot_uniform'))
model.add(Dense(1, activation='sigmoid'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          9889500   
                                                                 
 dropout (Dropout)           (None, 300, 300)          0         
                                                                 
 gru (GRU)                   (None, 512)               1250304   
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 11,140,317
Trainable params: 1,250,817
Non-trainable params: 9,889,500
_________________________________________________________________


In [10]:
#OPTIMIZATION
callbacks = [ tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
#TRAINING
%%time
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/10
36/36 [==============================] - 32s 768ms/step - loss: 0.5856 - accuracy: 0.6868 - val_loss: 0.5463 - val_accuracy: 0.7150 - lr: 0.0010
Epoch 2/10
36/36 [==============================] - 29s 818ms/step - loss: 0.5534 - accuracy: 0.7136 - val_loss: 0.5388 - val_accuracy: 0.7220 - lr: 0.0010
Epoch 3/10
36/36 [==============================] - 28s 791ms/step - loss: 0.5381 - accuracy: 0.7246 - val_loss: 0.5330 - val_accuracy: 0.7295 - lr: 0.0010
Epoch 4/10
36/36 [==============================] - 29s 794ms/step - loss: 0.5260 - accuracy: 0.7314 - val_loss: 0.5293 - val_accuracy: 0.7330 - lr: 0.0010
Epoch 5/10
36/36 [==============================] - 29s 800ms/step - loss: 0.5215 - accuracy: 0.7343 - val_loss: 0.5265 - val_accuracy: 0.7330 - lr: 0.0010
Epoch 6/10
36/36 [==============================] - 29s 810ms/step - loss: 0.5135 - accuracy: 0.7407 - val_loss: 0.5302 - val_accuracy: 0.7312 - lr: 0.0010
Epoch 7/10
36/36 [==============================] - 28s 792ms/st

In [12]:
#TESTING
score = model.evaluate(x_test, y_test, batch_size=batch_size)

print("ACCURACY:",score[1])
print("LOSS:",score[0])

acc = history.history['accuracy']
print("acc:", acc)
val_acc = history.history['val_accuracy']
print("val_acc:", val_acc)
loss = history.history['loss']
print("loss:", loss)
val_loss = history.history['val_loss']
print("val_loss:", val_loss)


10/10 [==============================] - 2s 216ms/step - loss: 0.5193 - accuracy: 0.7396
ACCURACY: 0.7396000027656555
LOSS: 0.5193184018135071
acc: [0.6868055462837219, 0.7135555744171143, 0.7246111035346985, 0.7313888669013977, 0.734250009059906, 0.7407222390174866, 0.7459999918937683, 0.7519999742507935, 0.757972240447998, 0.7623888850212097]
val_acc: [0.7149999737739563, 0.722000002861023, 0.7294999957084656, 0.7329999804496765, 0.7329999804496765, 0.731249988079071, 0.7337499856948853, 0.7272499799728394, 0.7277500033378601, 0.721750020980835]
loss: [0.5856441855430603, 0.553357720375061, 0.5381064414978027, 0.52604740858078, 0.5214840769767761, 0.5134782195091248, 0.5067764520645142, 0.49766790866851807, 0.490928053855896, 0.48387381434440613]
val_loss: [0.546291708946228, 0.5387861728668213, 0.5330337285995483, 0.529312252998352, 0.5265133380889893, 0.5302075743675232, 0.5289199352264404, 0.5322006344795227, 0.533134400844574, 0.536101222038269]


In [13]:
#SAVING 
save_dir = '/content/drive/MyDrive/nns/saved_nn_models/'
model_name = "RNN"
model.save(save_dir+model_name+".h5")


In [14]:
!ls /content/drive/MyDrive/nns/saved_nn_models/

FFNN1.h5  FFNN3.h5  FFNN5.h5  LSTM.h5  RNN.h5
